In [1]:
# def recursive_extract(term, results=None):
#     """Recursively extract all terms from a complex Prolog structure."""
#     if results is None:
#         results = []
    
#     # If this isn't a Term, we can't process it
#     if not isinstance(term, Term):
#         return results
    
#     # Check if this is a conjunction (,) or disjunction (;)
#     if term.functor == ',' or term.functor == ';':
#         # Recursively process each argument
#         for arg in term.args:
#             recursive_extract(arg, results)
#     # If this is a rule (:-), process both head and body
#     elif term.functor == ':-' and term.arity == 2:
#         # Add the head to results (usually not a langda, but might be useful)
#         recursive_extract(term.args[0], results)
#         # Process the body recursively
#         recursive_extract(term.args[1], results)
#     else:
#         # This is a regular predicate term
#         results.append(term)
    
#     return results

# import problog
# import re
# from typing import List
# from problog.logic import Term

# class Langda(object):
#     def __init__(self, model_string, caching=False, saving=False):
#         self.model_string = self.parse(model_string)

#     def parse(self, model_string):
#         langda_string, out_model_string = list(), list()
#         parser = problog.parser.PrologParser(problog.program.ExtendedPrologFactory())
#         langda_pattern = r'langda\("([^"\\]*(\\.[^"\\]*)*?)"\)'
        
#         for line in model_string.split('\n'):
#             if 'langda' in line:
#                 # matches = list(re.finditer(langda_pattern, line))
#                 modified_line = re.sub(langda_pattern, "\n{LANGDA}\n", line)
#                 out_model_string.append(modified_line)
#                 # print(matches)
#                 # print(re.finditer(langda_pattern, line))
#                 # if matches:
#                 #     modified_line = line
#                 #     for match in matches:
#                 #         modified_line = modified_line.replace(match.group(0), "\n{LANGDA}\n")

#                 #         # print("match:",match.group(1))
#                 #         out_model_string.append(modified_line)

#             else:
#                 out_model_string.append(line)
    
#         return '\n'.join(out_model_string)

# l = Langda(model_string)
# print(l.model_string)

In [ ]:
# model_string = """
# langda("hello; there").
# vols(X) :-
#     (langda("nothing"), langda("Is this, this, as ggod?! oh no- have fun"), where(X,Y));house(X).
# valid_drone_flight(X) :-
#     what(X);langda("there's a high building at 0.11"), vlos(X),
#     here(X).
# % Definition of a valid mission
# landscape(X) :- 
#     (langda("stop from there"), vlos(X)), addtion(X), langda("danger, stop!"), langda("what the hell?").
# final(X) :- langda("23 is a good number; stop here").
# """



In [11]:
text = """
Here's the code to replace the `{LANGDA}` placeholder, which adds the construction site avoidance rule:

```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
This code:
1. Ensures the drone maintains at least 20 units distance from the construction site at (500,500)
2. Combines with both possible valid flight conditions (VLOS with weight check or permits)
3. Maintains all other existing safety requirements
4. Uses the same structure as the rest of the rules with semicolon-separated alternatives



"""
import re

generated_codes = []
pattern = r"```(?:prolog|[a-z]*)?\n(.*?)```"
matches = re.findall(pattern, text, re.DOTALL)
for match in matches:
    generated_codes.append(match)
print(len(generated_codes))

3


In [12]:
pattern = r"\{\{LANGDA\}\}"
model_string = """
{{LANGDA}}
vols(X) :-
    {{LANGDA}}
    , langda("Is this, this, as ggod?! oh no- have fun"), where(X,Y));house(X).
valid_drone_flight(X) :-
    what(X);langda("there's a high building at 0.11"), vlos(X),
    here(X).
% Definition of a valid mission
landscape(X) :- 
    (langda("stop from there"), vlos(X)), addtion(X), langda("danger, stop!"), langda("what the hell?").
final(X) :- langda("23 is a good number; stop here").
"""

repl_iter = iter(generated_codes)
res = re.sub(r"\{\{LANGDA\}\}", lambda _: next(repl_iter, "{{LANGDA}}"), model_string)

In [7]:
import problog
from problog.logic import Term, Constant, Var
from problog.evaluator import Semiring
import torch
import numpy as np
import os
import sys

project_root = os.path.abspath("/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.deepprobcep.model import Model
from src.deepprobcep.network import Network
from src.deepprobcep.standard_networks import FC

from src.deepprobcep.gradient_semiring import SemiringGradient
from src.deepprobcep.vector_shape import VectorShape

# Define a simple test function for the network
def test_nn(network, *inputs):
    return torch.FloatTensor([0.3, 0.7])  # Example probabilities for binary classification

# Create a simple model
model_string = "nn(test_net,[X],Y,[a,b]) :: test_pred(X,Y).\nquery(X) :- test_pred(X,a)."
fc = FC(1, 2)  # Simple fully connected network with 1 input and 2 outputs
net = Network(fc, 'test_net', test_nn)
model = Model(model_string, [net])
# Create a query
query = Term('query', Constant(5))

# Get the compiled SDD and shape
sdd, shape = model.get_sdd(query)

# Examine the shape
print("Shape length:", shape.length)
print("Shape indices:", shape.indices)
print("Shape networks:", shape.networks)
print("Shape parameters:", shape.parameters)

# Create the semiring
semiring = SemiringGradient(model, shape)

# Print semiring methods
print("\nSemiring zero:", semiring.zero())
print("Semiring one:", semiring.one())

# Create a valid list term for testing
# This is how ProbLog represents lists internally: .(head, tail)
empty_list = Term('[]')
list_term = Term('.', Constant(5), empty_list)  # A list with just one element: [5]

# Now create a valid nn term with the proper list format
test_term = Term('nn', Constant('test_net'), list_term, Constant('a'))

try:
    prob_value = semiring.value(test_term)
    print("\nProbability value for test_term:", prob_value)
except Exception as e:
    print("\nError evaluating test_term:", e)
    
    # Let's try to debug what's happening
    from logic import term2list2
    try:
        print("Converting list term to Python list:", term2list2(list_term))
    except Exception as e:
        print("Error converting list term:", e)

# Test plus method with valid values
a = semiring.zero()
b = semiring.one()
plus_result = semiring.plus(a, b)
print("\nPlus result:", plus_result)

# Test times method
times_result = semiring.times(a, b)
print("\nTimes result:", times_result)

# Let's also try examining the SDD structure
print("\nSDD structure:")
print(sdd)

# Execute the query to see what happens
try:
    result = model.solve(query)
    print("\nQuery result:", result)
    if query in result:
        print("\nGradients:", result[query][1])
except Exception as e:
    print("\nError solving query:", e)

Shape length: 2
Shape indices: [0]
Shape networks: [('test_net', [5])]
Shape parameters: []

Semiring zero: (0.0, array([0., 0.]))
Semiring one: (1.0, array([0., 0.]))

Probability value for test_term: (0.30000001192092896, array([1., 0.]))

Plus result: (1.0, array([0., 0.]))

Times result: (0.0, array([0., 0.]))

SDD structure:
1: atom(identifier=(12, (5,) {{}}, 0), probability=nn(test_net,[5],a), group=(12, (5,) {{}}), name=query(5), source=None, is_extra=False)
Queries : 
* query(5) : 1 [query]


Query result: {query(5): (0.30000001192092896, {('test_net', [5]): array([1., 0.])})}

Gradients: {('test_net', [5]): array([1., 0.])}
